In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('asset/model_use/fresh_data.csv')
df_sample = data.filter(['tidy_text','tone'])
df_sample['label']= np.where(df_sample['tone']=='negative', 0, 1)
df=df_sample.filter(['tidy_text','label'])

# Extracting Features from Cleaned Tweets

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim

In [4]:
df['tidy_text']=df['tidy_text'].astype('str')
tokenized_text = df['tidy_text'].apply(lambda x: x.split())

In [5]:
#Doc2Vec Embedding
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models.doc2vec import LabeledSentence

C:\Users\Think\A3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
def add_label(twt):
    output = []
    for i, s in zip(twt.index, twt):
        output.append(LabeledSentence(s, ["tweet_" + str(i)]))
    return output

In [7]:
labeled_texts = add_label(tokenized_text) # label all the tweets

C:\Users\Think\A3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  after removing the cwd from sys.path.


In [8]:
model_d2v = gensim.models.Doc2Vec(dm=1, # dm = 1 for ‘distributed memory’ model 
                                  dm_mean=1, # dm = 1 for using mean of the context word vectors
                                  size=100, 
                                  window=5, 
                                  negative=7,
                                  min_count=5,
                                  workers=3, 
                                  alpha=0.1, 
                                  seed = 23)

model_d2v.build_vocab([i for i in tqdm(labeled_texts)])

C:\Users\Think\A3\lib\site-packages\gensim\models\doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 4001/4001 [00:00<00:00, 667226.36it/s]


In [9]:
docvec_arrays = np.zeros((len(tokenized_text), 100))

for i in range(len(df)):
    docvec_arrays[i,:] = model_d2v.docvecs[i].reshape((1,100))
    
docvec_df = pd.DataFrame(docvec_arrays)
docvec_df.shape

(4001, 100)

# Model Building (Logistic Regression)

In [10]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,mean_squared_error

In [11]:
train_d2v = docvec_df.iloc[:2500,:]
valid_d2v = docvec_df.iloc[2500:,:]

In [12]:
xtrain_d2v, xtest_d2v, ytrain, yvalid = train_test_split(train_d2v, df['label'][:2500],  
                                                          random_state=42, 
                                                          test_size=0.3)

In [13]:
model= LogisticRegression()
model.fit(xtrain_d2v, ytrain)

prediction = model.predict_proba(xtest_d2v)
prediction_int = prediction[:,1] >= 0.3
prediction_int = prediction_int.astype(np.int)
f1_score=f1_score(yvalid, prediction_int)

In [14]:
predictions = model.predict(xtest_d2v)
MSE = mean_squared_error(yvalid, predictions)
r2 = model.score(xtest_d2v, yvalid)

In [15]:
print(f"MSE: {MSE}, R2: {r2}, F1_score:{f1_score}")

MSE: 0.232, R2: 0.768, F1_score:0.8687782805429864


In [16]:
validation=df[2500:]
val_pre=model.predict_proba(valid_d2v)[:,1] >= 0.3
validation['pre_tone']=val_pre.astype(np.int)

C:\Users\Think\A3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
from collections import Counter 

ewr=validation['label']==validation['pre_tone']
Counter(ewr)

Counter({True: 1074, False: 427})

In [18]:
1074/1501

0.715522984676882